In [1]:
from data_loader import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
train = load_train_data('data/train.csv', is_df = False)
# test = load_test_data('data/test.csv', 'data/test_labels.csv', is_df = False)

Using TensorFlow backend.
/Users/caojingchen/UCI/projects/data_mining/toxic/data_loader.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return train_df['id'].tolist(), train_df['comment_text'].tolist(), train_df.iloc[:, 2:].as_matrix(),\
/Users/caojingchen/UCI/projects/data_mining/toxic/data_loader.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ), valid_df.iloc[:, 2:].as_matrix()


In [5]:
X, y = train[1], train[2]
tok = Tokenizer(num_words = 20000)
tok.fit_on_texts(list(X))
X = tok.texts_to_sequences(X)
X = sequence.pad_sequences(X, maxlen = 150)
# test = tok.texts_to_sequences(test)
# X_test = sequence.pad_sequences(test, maxlen = 150)

In [8]:
inp = Input(shape = (150, ))
layer = Embedding(20000, 128)(inp)
layer = Bidirectional(LSTM(50, return_sequences = True, recurrent_dropout = 0.15))(layer)
layer = GlobalMaxPool1D()(layer)
layer = Dropout(0.2)(layer)
layer = Dense(50, activation = 'relu')(layer)
layer = Dropout(0.2)(layer)
layer = Dense(6, activation = 'sigmoid')(layer)
model = Model(inputs = inp, outputs = layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
file_path = 'save_best'
checkpoint = ModelCheckpoint(file_path, monitor = 'val_loss', verbose = 1, save_best_only=True)
early_stop = EarlyStopping(monitor = 'val_loss', patience = 1)

In [10]:
hist = model.fit(X, y, batch_size = 32, epochs = 2, validation_split = 0.2, callbacks = [checkpoint, early_stop])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 114916 samples, validate on 28729 samples
Epoch 1/2
 25728/114916 [=====>........................] - ETA: 8:34 - loss: 0.1236 - acc: 0.9638

KeyboardInterrupt: 

In [ ]:
vloss = hist.history['val_loss']
loss = hist.history['loss']

x_len = np.arange(len(loss))

plt.plot(x_len, vloss, marker='.', c='red', label='vloss')
plt.plot(x_len, loss, marker='.', c='blue', label='loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()